In [1]:
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf

import warnings
warnings.filterwarnings('ignore')

In [2]:
datapath = '/Users/yufanwang/Desktop/MarmosetWM_Project/revision_NC/af_projection/result'

In [ ]:
human_af_l_all = np.loadtxt(f'{datapath}/probtrackx2/human_af_l_thr0_norm.txt')
chimp_af_l_all = np.loadtxt(f'{datapath}/probtrackx2/chimp_af_l_thr0_norm.txt')
macaque_af_l_all = np.loadtxt(f'{datapath}/probtrackx2/macaque_tvb_af_l_thr0_norm.txt')
marmoset_af_l_all = np.loadtxt(f'{datapath}/probtrackx2/marmoset_MBM_af_l_thr0_norm.txt')

human_af_r_all = np.loadtxt(f'{datapath}/probtrackx2/human_af_r_thr0_norm.txt')
chimp_af_r_all = np.loadtxt(f'{datapath}/probtrackx2/chimp_af_r_thr0_norm.txt')
macaque_af_r_all = np.loadtxt(f'{datapath}/probtrackx2/macaque_tvb_af_r_thr0_norm.txt')
marmoset_af_r_all = np.loadtxt(f'{datapath}/probtrackx2/marmoset_MBM_af_r_thr0_norm.txt')

In [ ]:
index_human_chimp_L = np.loadtxt(f'/Users/yufanwang/Desktop/MarmosetWM_Project/revision_NC/indi_bp_atlas/index_human_chimp_L.txt')
index_human_macaque_L = np.loadtxt(f'/Users/yufanwang/Desktop/MarmosetWM_Project/revision_NC/indi_bp_atlas/index_human_macaque_L.txt')
index_human_marmoset_L = np.loadtxt(f'/Users/yufanwang/Desktop/MarmosetWM_Project/revision_NC/indi_bp_atlas/index_human_marmoset_L.txt')

## construct data

In [5]:
def read_subject_list(txt_path):
    with open(txt_path, "r") as f:
        subs = [line.strip() for line in f if line.strip()]
    return subs

def build_species_volume_df(list_path, species_name, left_vals, right_vals, region, volume_path):
    subj = read_subject_list(list_path)
    L = np.asarray(left_vals).reshape(-1)
    R = np.asarray(right_vals).reshape(-1)

    if not (len(subj) == len(L) == len(R)):
        raise ValueError(
            f"[{species_name}] length mismatch: "
            f"subjects={len(subj)}, L={len(L)}, R={len(R)}"
        )

    vol = np.loadtxt(volume_path)
    vol_wb, vol_gm, vol_wm = vol[:, 0], vol[:, 1], vol[:, 2]

    df_L = pd.DataFrame({
        "subject_id": subj,
        "species": species_name,
        "hemisphere": "LH",
        "region": region,
        "volume_wb": vol_wb,
        "volume_gm": vol_gm,
        "volume_wm": vol_wm,
        "connectivity_score": L
    })

    df_R = pd.DataFrame({
        "subject_id": subj,
        "species": species_name,
        "hemisphere": "RH",
        "region": region,
        "volume_wb": vol_wb,
        "volume_gm": vol_gm,
        "volume_wm": vol_wm,
        "connectivity_score": R
    })

    return pd.concat([df_L, df_R], ignore_index=True)

In [6]:
human_list_path = "/Users/yufanwang/Desktop/MarmosetWM_Project/revision_NC/indi_bp_atlas/humanlist40.txt"
chimp_list_path = "/Users/yufanwang/Desktop/MarmosetWM_Project/revision_NC/indi_bp_atlas/chimplist46.txt"
macaque_list_path = "/Users/yufanwang/Desktop/MarmosetWM_Project/revision_NC/indi_bp_atlas/macaque_tvb_list.txt"
marmoset_list_path = "/Users/yufanwang/Desktop/MarmosetWM_Project/revision_NC/indi_bp_atlas/marmoset_MBM_list.txt"

i = 17
region = "A45c"

roi_index_human = i - 1
roi_index_chimp = index_human_chimp_L[roi_index_human] - 1
roi_index_macaque = index_human_macaque_L[roi_index_human] - 1
roi_index_marmoset = index_human_marmoset_L[roi_index_human] - 1

value_human_L = human_af_l_all[:,roi_index_human]
value_chimp_L = chimp_af_l_all[:,int(roi_index_chimp)]
value_macaque_L = macaque_af_l_all[:,int(roi_index_macaque)]
value_marmoset_L = marmoset_af_l_all[:,int(roi_index_marmoset)]

value_human_R = human_af_r_all[:,roi_index_human]
value_chimp_R = chimp_af_r_all[:,int(roi_index_chimp)]
value_macaque_R = macaque_af_r_all[:,int(roi_index_macaque)]
value_marmoset_R = marmoset_af_r_all[:,int(roi_index_marmoset)]

volume_human_path = '/Users/yufanwang/Desktop/MarmosetWM_Project/revision_NC/volume/volume_human.txt'
volume_chimp_path = '/Users/yufanwang/Desktop/MarmosetWM_Project/revision_NC/volume/volume_chimp.txt'
volume_macaque_path = '/Users/yufanwang/Desktop/MarmosetWM_Project/revision_NC/volume/volume_macaque.txt'
volume_marmoset_path = '/Users/yufanwang/Desktop/MarmosetWM_Project/revision_NC/volume/volume_marmoset.txt'

df = pd.concat([
    build_species_volume_df(human_list_path,    "human",      value_human_L,    value_human_R, region, volume_human_path),
    build_species_volume_df(chimp_list_path,    "chimpanzee", value_chimp_L,    value_chimp_R, region, volume_chimp_path),
    build_species_volume_df(macaque_list_path,  "macaque",    value_macaque_L,  value_macaque_R, region, volume_macaque_path),
    build_species_volume_df(marmoset_list_path, "marmoset",   value_marmoset_L, value_marmoset_R, region, volume_marmoset_path),
], ignore_index=True)

print(df.head())
print(df.shape)  # 应该是 2 * (40+46+8+24) = 236 行

df.to_csv(f'{datapath}/af_projection_{region}_volume.csv', index=False)

  subject_id species hemisphere region  volume_wb  volume_gm  volume_wm  \
0     100307   human         LH   A45c  1130925.0   656756.0   461873.0   
1     100408   human         LH   A45c  1304369.0   716157.0   567342.0   
2     103414   human         LH   A45c  1082325.0   619604.0   449285.0   
3     105115   human         LH   A45c  1228840.0   693113.0   483771.0   
4     106016   human         LH   A45c  1115450.0   660915.0   442756.0   

   connectivity_score  
0            0.335231  
1            0.339382  
2            0.133474  
3            0.264675  
4            0.541276  
(236, 8)
